# Mathematical concepts

<center>
<img src="ComposableNumberTree.png" width="680" height="400">
</center>

 <pre>
A001477  (integers)
├── {0, 1}
├── A000040  (primes)
│   └── A392439  (composable-preceded primes)
└── A002808  (composites)
    ├── A392440  (composable)
    │   ├── A392438  (2-composable)
    │   └── A392437  (3-composable)
    └── A392500  (noncomposable)
        ├── A392498  (sparse)
        │   ├── A392493  (2-sparse)
        │   └── A392494  (3-sparse)
        └── A392499  (rigid)
            ├── A392496  (2-rigid)
            └── A392495  (3-rigid)
</pre>

In [42]:
from math import isqrt

In [43]:
def smallest_prime_divisor(n: int) -> int:
    if n % 2 == 0:
        return 2
    for d in range(3, isqrt(n) + 1, 2):
        if n % d == 0:
            return d
    return n  # n is prime

In [44]:
def nontrivial_divisors(n: int) -> list[int]:
    """Divisors d with 1 < d < n"""
    divs = set()
    for d in range(2, isqrt(n) + 1):
        if n % d == 0:
            divs.add(d)
            q = n // d
            if q != d:
                divs.add(q)
    return sorted(divs)

In [45]:
def D_p(n: int) -> tuple[int, list[int]]:
    """
    Returns (L, D_p(n)) where
    L = n / spf(n)
    D_p(n) = { d | 1 < d < L }
    """
    p = smallest_prime_divisor(n)
    L = n // p
    D = [d for d in nontrivial_divisors(n) if d < L]
    return L, D

In [46]:
def subset_sum_exists(nums: list[int], target: int) -> bool:
    bits = 1  # bit i means sum i reachable
    for x in nums:
        bits |= bits << x
        if bits >> target & 1:
            return True
    return False

In [47]:
def is_composable(n: int) -> bool:
    if n < 4:
        return False
    p = smallest_prime_divisor(n)
    if p == n:
        return False
    L, D = D_p(n)
    return subset_sum_exists(D, L)

In [48]:
def is_rigid(n: int) -> bool:
    if n < 4:
        return False
    p = smallest_prime_divisor(n)
    if p == n:
        return False
    L, D = D_p(n)
    return sum(D) >= L and not subset_sum_exists(D, L)

In [49]:
def is_sparse(n: int) -> bool:
    if n < 4:
        return False
    p = smallest_prime_divisor(n)
    if p == n:
        return False
    L, D = D_p(n)
    return sum(D) < L

In [50]:
def is_noncomposable(n: int) -> bool:
    return is_sparse(n) or is_rigid(n)

In [51]:
def composable_preceded_primes(limit: int) -> list[int]:
    primes = []
    for n in range(4, limit):
        if is_composable(n):
            p = n + 1
            if smallest_prime_divisor(p) == p:
                primes.append(p)
    return primes

### p-class-based

In [52]:
def is_p_composable(n: int, p: int) -> bool:
    return smallest_prime_divisor(n) == p and is_composable(n)

In [53]:
def composable_list(p: int, limit: int) -> list[int]:
    return [n for n in range(p*p, limit+1, p) if is_p_composable(n, p)]

In [54]:
def is_p_rigid(n: int, p: int) -> bool:
    return smallest_prime_divisor(n) == p and is_rigid(n)

In [55]:
def rigid_list(p: int, limit: int) -> list[int]:
    return [n for n in range(p*p, limit+1, p) if is_p_rigid(n, p)]

In [56]:
def is_p_sparse(n: int, p: int) -> bool:
    return smallest_prime_divisor(n) == p and is_sparse(n)

In [57]:
def sparse_list(p: int, limit: int) -> list[int]:
    return [n for n in range(p*p, limit+1, p) if is_p_sparse(n, p)]

### One-pass classifier

In [58]:
from math import isqrt

def classify_number(n: int) -> str:
    """
    Classifies n >= 0 into exactly one of:
    'idempotent', 'prime', 'sparse', 'rigid', or 'composable'. 
    """
    if n < 2:
        return "idempotent"

    # smallest prime divisor
    if n % 2 == 0:
        p = 2
    else:
        p = None
        for d in range(3, isqrt(n) + 1, 2):
            if n % d == 0:
                p = d
                break
        if p is None:
            return "prime"

    if p == n:
        return "prime"

    # compute L and D_p(n)
    L = n // p
    D = []
    for d in range(2, isqrt(n) + 1):
        if n % d == 0:
            q = n // d
            if d < L:
                D.append(d)
            if q != d and q < L:
                D.append(q)

    # sparse test (cheap, do first)
    total = sum(D)
    if total < L:
        return "sparse"

    # subset-sum test (bitset)
    bits = 1
    for x in D:
        bits |= bits << x
        if (bits >> L) & 1:
            return "composable"

    return "rigid"

In [59]:
for n in range(13):
    print(f"{n:>3} | {classify_number(n)}")

  0 | idempotent
  1 | idempotent
  2 | prime
  3 | prime
  4 | sparse
  5 | prime
  6 | sparse
  7 | prime
  8 | sparse
  9 | sparse
 10 | sparse
 11 | prime
 12 | composable


In [60]:
def first_p_composable(p: int, limit: int = 10**12) -> int | None:
    # Optimization: Step by p if p=2, else step by 2p to skip even numbers
    step = p if p == 2 else 2 * p
    for n in range(p*p, limit + 1, step):
        if smallest_prime_divisor(n) != p: continue
        L = n // p
        D = [d for d in nontrivial_divisors(n) if 1 < d < L]
        has_sum = subset_sum_exists(D, L)
        if has_sum:
            return n

In [61]:
def first_p_rigid(p: int, limit: int = 10**12) -> int | None:
    for n in range(p*p, limit+1, p):
        if smallest_prime_divisor(n) != p: continue
        L = n // p
        D = [d for d in nontrivial_divisors(n) if 1 < d < L]
        if sum(D) >= L and not subset_sum_exists(D, L):
            return n
    return None

In [62]:
def first_p_sparse(p: int, limit: int = 1) -> int:
    return p*p

# Math terms → OEIS speak

In [63]:
TEST = True

## Composable numbers

In [64]:
def A392440_list(upto: int) -> list[int]:
    """
    Composable numbers 
    """
    return [n for n in range(2, upto + 1) if is_composable(n)]


if TEST: 
    print(A392440_list(252))

[12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, 78, 80, 84, 90, 96, 100, 102, 108, 112, 114, 120, 126, 132, 135, 138, 140, 144, 150, 156, 160, 162, 168, 174, 176, 180, 186, 192, 196, 198, 200, 204, 208, 210, 216, 220, 222, 224, 225, 228, 234, 240, 246, 252]


In [65]:
def A392440_p(p: int, upto: int) -> list[int]:
    """
    Classified composable
    """
    return composable_list(p, upto)


if TEST: 
    print(A392440_p(2, 50))
    print(A392440_p(3, 500))
    print(A392440_p(5, 4025))
    print(A392440_p(7, 20111))
    print(A392440_p(11, 62491))

[12, 18, 24, 30, 36, 40, 42, 48]
[135, 225, 315, 345, 405, 495]
[1645, 1925, 2275, 3185, 4025]
[14651, 17017, 19019, 20111]
[8041, 23881, 46189, 62491]


In [66]:
def A392438_list(upto: int) -> list[int]:
    """
    2-composable numbers
    """
    return composable_list(2, upto)


if TEST: 
    print(A392438_list(260))

[12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, 78, 80, 84, 90, 96, 100, 102, 108, 112, 114, 120, 126, 132, 138, 140, 144, 150, 156, 160, 162, 168, 174, 176, 180, 186, 192, 196, 198, 200, 204, 208, 210, 216, 220, 222, 224, 228, 234, 240, 246, 252, 258, 260]


In [67]:
def A392437_list(upto: int) -> list[int]:
    """
    3-composable numbers
    """
    return composable_list(3, upto)


if TEST: 
    print(A392437_list(2583))

[135, 225, 315, 345, 405, 495, 525, 585, 675, 765, 855, 945, 975, 1035, 1125, 1155, 1197, 1215, 1305, 1365, 1395, 1425, 1485, 1575, 1617, 1665, 1725, 1755, 1785, 1845, 1935, 1953, 1995, 2025, 2079, 2115, 2145, 2205, 2295, 2331, 2385, 2415, 2457, 2475, 2565, 2583]


## Sparse numbers

In [68]:
def A392498_list(upto: int) -> list[int]:
    """
    Sparse numbers
    """
    return [n for n in range(2, upto + 1) if is_sparse(n)]


if TEST: 
    print(A392498_list(125))

[4, 6, 8, 9, 10, 14, 15, 16, 21, 22, 25, 26, 27, 28, 32, 33, 34, 35, 38, 39, 44, 46, 49, 50, 51, 52, 55, 57, 58, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 81, 82, 85, 86, 87, 91, 92, 93, 94, 95, 98, 99, 106, 110, 111, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125]


In [69]:
def A392498_p(p: int, upto: int) -> list[int]:
    """
    Classified sparse
    """
    return sparse_list(p, upto)


if TEST: 
    print(A392498_p(2, 20))
    print(A392498_p(3, 40))
    print(A392498_p(5, 100))
    print(A392498_p(7, 200))
    print(A392498_p(11, 320))

[4, 6, 8, 10, 14, 16]
[9, 15, 21, 27, 33, 39]
[25, 35, 55, 65, 85, 95]
[49, 77, 91, 119, 133, 161]
[121, 143, 187, 209, 253, 319]


In [70]:
def A392493_list(upto: int) -> list[int]:
    """
    2-sparse numbers
    """
    return sparse_list(2, upto)


if TEST: 
    print(A392493_list(172))

[4, 6, 8, 10, 14, 16, 22, 26, 28, 32, 34, 38, 44, 46, 50, 52, 58, 62, 64, 68, 74, 76, 82, 86, 92, 94, 98, 106, 110, 116, 118, 122, 124, 128, 130, 134, 136, 142, 146, 148, 152, 154, 158, 164, 166, 170, 172]


In [71]:
def A392494_list(upto: int) -> list[int]:
    """
    3-sparse numbers
    """
    return sparse_list(3, upto)


if TEST: 
    print(A392494_list(411))

[9, 15, 21, 27, 33, 39, 51, 57, 63, 69, 75, 81, 87, 93, 99, 111, 117, 123, 129, 141, 147, 153, 159, 171, 177, 183, 201, 207, 213, 219, 231, 237, 243, 249, 261, 267, 273, 279, 291, 297, 303, 309, 321, 327, 333, 339, 351, 357, 363, 369, 375, 381, 387, 393, 399, 411]


## Rigid numbers

In [72]:
def A392499_list(upto: int) -> list[int]:
    """
    Rigid numbers
    """
    return [n for n in range(2, upto + 1) if is_rigid(n)]


if TEST: 
    print(A392499_list(1504))

[20, 45, 70, 88, 104, 105, 165, 175, 189, 195, 255, 272, 285, 304, 350, 368, 385, 441, 455, 464, 567, 572, 595, 650, 665, 693, 715, 735, 805, 819, 825, 836, 875, 1001, 1015, 1071, 1085, 1184, 1225, 1275, 1287, 1295, 1309, 1312, 1323, 1376, 1435, 1449, 1463, 1504]


In [73]:
def A392499_p(p: int, upto: int) -> list[int]:
    """
    Classified rigid
    """
    return rigid_list(p, upto)


if TEST: 
    print(A392499_p(2, 305))
    print(A392499_p(3, 255))
    print(A392499_p(5, 715))
    print(A392499_p(7, 1729))
    print(A392499_p(11, 3553))

[20, 70, 88, 104, 272, 304]
[45, 105, 165, 189, 195, 255]
[175, 385, 455, 595, 665, 715]
[1001, 1309, 1463, 1547, 1729]
[1573, 2431, 2717, 3289, 3553]


In [74]:
def A392496_list(upto: int) -> list[int]:
    """
    2-rigid numbers
    """
    return rigid_list(2, upto)


if TEST: 
    print(A392496_list(9730))

[20, 70, 88, 104, 272, 304, 350, 368, 464, 572, 650, 836, 1184, 1312, 1376, 1504, 1696, 1888, 1952, 3770, 4030, 4288, 4544, 4672, 5056, 5312, 5696, 5704, 5810, 5830, 6208, 6464, 6592, 6790, 6808, 6848, 6976, 7144, 7192, 7232, 7630, 7910, 7912, 8024, 8056, 9272, 9590, 9730]


In [75]:
def A392495_list(upto: int) -> list[int]:
    """
    3-rigid numbers
    """
    return rigid_list(3, upto)


if TEST: 
    print(A392495_list(5607))

[45, 105, 165, 189, 195, 255, 285, 441, 567, 693, 735, 819, 825, 1071, 1275, 1287, 1323, 1449, 1683, 1701, 1815, 1827, 1911, 2175, 2325, 2535, 2541, 2709, 2775, 3075, 3087, 3339, 3525, 3717, 3843, 3975, 4221, 4335, 4473, 4575, 5025, 5103, 5229, 5325, 5475, 5607]


## Noncomposable numbers

In [76]:
def A392500_list(upto: int) -> list[int]:
    """
    noncomposable numbers
    """
    return [n for n in range(2, upto + 1) if is_noncomposable(n)]

if TEST: 
    print(A392500_list(118))

[4, 6, 8, 9, 10, 14, 15, 16, 20, 21, 22, 25, 26, 27, 28, 32, 33, 34, 35, 38, 39, 44, 45, 46, 49, 50, 51, 52, 55, 57, 58, 62, 63, 64, 65, 68, 69, 70, 74, 75, 76, 77, 81, 82, 85, 86, 87, 88, 91, 92, 93, 94, 95, 98, 99, 104, 105, 106, 110, 111, 115, 116, 117, 118]


## Composable-preceded primes

In [77]:
def A392439_list(limit: int) -> list[int]:
    """
    composable-preceded primes
    """
    return composable_preceded_primes(limit)


if TEST: 
    print(A392439_list(521))

[13, 19, 31, 37, 41, 43, 61, 67, 73, 79, 97, 101, 103, 109, 113, 127, 139, 151, 157, 163, 181, 193, 197, 199, 211, 223, 229, 241, 271, 277, 281, 283, 307, 313, 331, 337, 349, 353, 367, 373, 379, 397, 401, 409, 421, 433, 439, 449, 457, 461, 463, 487, 491, 499, 521]


## Smallest term in p-class

In [78]:
from sympy import sieve

In [79]:
def A392492_list(upto: int) -> list[int | None]:
    """
    Smallest p-composable number
    """
    return [first_p_composable(p) for p in sieve.primerange(2, upto)]


if TEST:  # this takes some time!
    print(A392492_list(36))

[12, 135, 1645, 14651, 8041, 175253, 115957, 695267, 2128351, 1590331, 5024263]


In [80]:
def A392497_list(upto: int) -> list[int | None]:
    """
    Smallest p-rigid number
    """
    return [first_p_rigid(p) for p in sieve.primerange(2, upto)]


if TEST: 
    print(A392497_list(150))

[20, 45, 175, 1001, 1573, 4199, 5491, 12673, 20677, 26071, 47027, 65231, 72283, 107113, 146969, 190747, 212341, 290177, 347261, 367993, 478661, 583573, 716539, 871933, 1009091, 1050703, 1201289, 1247941, 1564259, 1879981, 2279269, 2494633, 2608891, 3127361, 3352351]


## Class Test

In [81]:
def test_p_class_completeness(p: int, limit: int) -> None:
    """
    Verifies that for a prime p, every composite n with spf(n) == p
    is either p-sparse, p-rigid, or p-composable.
    """
    # Gather all composites in the p-class
    # (multiples of p where p is the smallest prime divisor)
    p_class_composites = []
    step = p if p == 2 else 2 * p
    start = p + step

    for n in range(start, limit + 1, step):
        if smallest_prime_divisor(n) == p:
            p_class_composites.append(n)

    # Get the lists 
    com = composable_list(p, limit)
    rig = rigid_list(p, limit)
    spa = sparse_list(p, limit )
    
    # Calculate the union
    union_count = len(com) + len(rig) + len(spa)
    total_expected = len(p_class_composites)

    # Reporting
    print(f"--- Completeness test for p={p} (up to {limit}) ---")
    print(f"Total composites in p-class: {total_expected}")
    print(f"  - sparse:     {len(spa):>6}")
    print(f"  - rigid:      {len(rig):>6}")
    print(f"  - composable: {len(com):>6}")
    print(f"Total categorized:     {union_count}")

    is_complete = union_count == total_expected
    print(f"Completeness verified: {is_complete} <- **** \n")
    

In [82]:
for p in [2, 3, 5, 7, 11, 13]:
    test_p_class_completeness(p, 10**5)

--- Completeness test for p=2 (up to 100000) ---
Total composites in p-class: 49999
  - sparse:      25414
  - rigid:         304
  - composable:  24281
Total categorized:     49999
Completeness verified: True <- **** 

--- Completeness test for p=3 (up to 100000) ---
Total composites in p-class: 16666
  - sparse:      13882
  - rigid:         541
  - composable:   2243
Total categorized:     16666
Completeness verified: True <- **** 

--- Completeness test for p=5 (up to 100000) ---
Total composites in p-class: 6666
  - sparse:       6024
  - rigid:         323
  - composable:    319
Total categorized:     6666
Completeness verified: True <- **** 

--- Completeness test for p=7 (up to 100000) ---
Total composites in p-class: 3808
  - sparse:       3655
  - rigid:         102
  - composable:     51
Total categorized:     3808
Completeness verified: True <- **** 

--- Completeness test for p=11 (up to 100000) ---
Total composites in p-class: 2077
  - sparse:       1972
  - rigid:       